In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

스태킹(Stacking)  
스태킹은 베이스 학습기와 메타 학습기로 구성되어 있고 베이스 학습기는와 메타 학습기는 서포트 벡터 머신, 랜덤 포레스트와 같은 학습 모델이다. 베이스 학습기가 먼저 학습한 후 메타 학습기는 베이스 학습기의 예측을 피쳐 데이터로 활용해 최종 예측을 한다.

스태킹 알고리즘을 활용해 암 여부를 예측하는 모델을 만든다.

In [2]:
# 데이터 불러오기
from sklearn import datasets # 사이킷런이 제공하는 데이터셋을 사용하기 위해 import 한다.
raw_cancer = datasets.load_breast_cancer() # 위스콘신 암 데이터를 불러온다.

In [3]:
# 피쳐, 타겟 데이터 저장
X = raw_cancer.data # 피쳐 데이터를 저장한다.
y = raw_cancer.target # 타겟 데이터를 저장한다.
X.shape

(569, 30)

In [4]:
# 트레이닝, 테스트 데이터 분할
from sklearn.model_selection import train_test_split # 트레이닝, 테스트 데이터 분할을 위해 import 한다.
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)
print(X_tn.shape, y_tn.shape, X_te.shape, y_te.shape) # 트레이닝 데이터와 테스트 데이터로 분할한다.

(426, 30) (426,) (143, 30) (143,)


In [5]:
# 데이터 표준화
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위해 import 한다.
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
# 표준화는 트레이닝 데이터를 기반으로 실행하므로 트레이닝 피쳐 데이터 X_tn을 표준화 스케일러에 적합시킨다.
X_tn_std = std_scale.fit_transform(X_tn) # 트레이닝 데이터를 적합 후 표준화 한다.
X_te_std = std_scale.transform(X_te) # 테스트 데이터를 표준화 한다.

In [10]:
# 모델 생성 후 데이터 학습
# 메타 학습기 - LogisticRegression, 베이스 학습기 - SVC, GaussianNB
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀 알고리즘을 사용하기 위해 import 한다.
from sklearn.svm import SVC # 서포트 벡터 머신 알고리즘을 사용하기 위해 import 한다.
from sklearn.naive_bayes import GaussianNB # 가우시안 나이브베이즈 알고리즘을 사용하기 위해 import 한다.
from sklearn.ensemble import StackingClassifier # 스태킹 알고리즘을 사용하기 위해 import 한다.

# 베이스 학습기를 만든다.
clf1 = SVC(kernel='linear') # 서포트 벡터 머신 알고리즘을 사용하는 베이스 학습기를 만든다.
clf2 = GaussianNB() # 가우시안 나이브베이즈 알고리즘을 사용하는 베이스 학습기를 만든다.
# estimators 옵션으로 스태킹 모델에서 사용할 베이스 학습기를 지정한다.
# final_estimator 옵션으로 스태킹 모델에서 사용할 메타 학습기를 지정한다.
clf_stk = StackingClassifier(estimators=[('svm', clf1), ('gnb', clf2)], 
                             final_estimator=LogisticRegression()) # 스태킹 모델을 만든다.
# 표준화된 트레이닝 피쳐 데이터 X_tn_std와 트레이닝 타겟 데이터 y_tn를 모델에 넣어서 스태킹 알고리즘을 학습시킨다.
clf_stk.fit(X_tn_std, y_tn)

StackingClassifier(estimators=[('svm', SVC(kernel='linear')),
                               ('gnb', GaussianNB())],
                   final_estimator=LogisticRegression())

In [11]:
# 테스트 데이터 예측
# 표준화된 테스트 데이터 X_te_std로 예측한다.
pred_stk= clf_stk.predict(X_te_std)
print(pred_stk)

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0]


In [12]:
# 정확도 평가
from sklearn.metrics import accuracy_score # 모델의 정확도를 평가하기 위해 import 한다.
# accuracy_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_stk)를 넘겨 정확도를 계산한다.
accuracy = accuracy_score(y_te, pred_stk)
print(accuracy)

0.965034965034965


In [13]:
# 정밀도 평가
from sklearn.metrics import precision_score # 모델의 정밀도를 평가하기 위해 import 한다.
# precision_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_stk)를 넘겨 정밀도를 계산한다.
precision = precision_score(y_te, pred_stk, average=None)
print(precision)

[0.96153846 0.96703297]


In [14]:
# 재현율 평가
from sklearn.metrics import recall_score # 모델의 재현율를 평가하기 위해 import 한다.
# recall_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_stk)를 넘겨 재현율를 계산한다.
recall = recall_score(y_te, pred_stk, average=None)
print(recall)

[0.94339623 0.97777778]


In [15]:
# f1 score 평가
from sklearn.metrics import f1_score # 모델의 f1 score를 평가하기 위해 import 한다.
# f1_score() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_stk)를 넘겨 f1 score를 계산한다.
f1 = f1_score(y_te, pred_stk, average=None)
print(f1)

[0.95238095 0.97237569]


In [16]:
# 혼돈 행렬 확인
from sklearn.metrics import confusion_matrix # 혼돈 행렬을 만들기 위해 import 한다.
# confusion_matrix() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_stk)를 넘겨 혼돈 행렬을 만든다.
conf_matrix = confusion_matrix(y_te, pred_stk)
print(conf_matrix)

[[50  3]
 [ 2 88]]


In [17]:
# 분류 리포트 확인
from sklearn.metrics import classification_report # 분류 리포트를 출력하기 위해 import 한다.
# classification_report() 메소드의 인수로 실제 타겟 데이터(y_te)와 예측된 데이터(pred_stk)를 넘겨 분류 리포트를 만든다.
target_names = raw_cancer.target_names
class_report = classification_report(y_te, pred_stk, target_names=target_names)
print(class_report)

              precision    recall  f1-score   support

   malignant       0.96      0.94      0.95        53
      benign       0.97      0.98      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143

